 <img src="https://fiverr-res.cloudinary.com/images/t_main1,q_auto,f_auto,q_auto,f_auto/attachments/delivery/asset/ff861c9745d910a54d4d61e4e37a1602-1605431705/SampleHands/create-personalized-among-us-gif-and-png-for-you.GIF"> 

# <p style = 'text-align: center; background-color:#f5f557;'> Among Us </p>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

users = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        users.append(os.path.join(dirname, filename))

user_ = pd.read_csv(users[0])
columns = list(user_.columns) + ['id']

user = pd.read_csv(users[0])
user['id'] = 0
user = user.values
for _,i in enumerate(users[1:], 1):
    new_user = pd.read_csv(i)
    new_user['id'] = _
    new_user = new_user.values
    user = np.vstack((user, new_user))
user = pd.DataFrame(user, columns = columns)

user['Task Completed'] = user['Task Completed'].apply(lambda x: np.nan if x == '-' else x).astype('float32')
user['All Tasks Completed'] = user['All Tasks Completed'].apply(lambda x: np.nan if x == '-' else x)
user['Murdered'] = user['Murdered'].apply(lambda x: np.nan if x == '-' else x)
user['Imposter Kills'] = user['Imposter Kills'].apply(lambda x: np.nan if x == '-' else x).astype('float32')
user['Sabotages Fixed'] = user['Sabotages Fixed'].astype('float32')
user['Rank Change'] = user['Rank Change'].fillna('No')
user['id'] = user['id'].astype('int32')
user['Game Completed Date'] = pd.to_datetime(user['Game Completed Date'], format = '%m/%d/%Y at %I:%M:%S %p EST')
user['Game Length'] = pd.to_datetime(user['Game Length'], format = '%Mm %Ss')
user['Time to complete all tasks'] = user['Time to complete all tasks'].apply(lambda x: np.nan if x == '-' else x)
user['Time to complete all tasks'] = pd.to_datetime(user['Time to complete all tasks'], format = '%Mm %Ss')

crewmate = user.query('''Team == "Crewmate"''').drop(['Team', 'Imposter Kills'], axis = 1)
imposter = user.query('''Team == "Imposter"''').loc[:,user.query('''Team == "Imposter"''').notna().all()].drop('Team', axis = 1)

In [ ]:
user_group = user.groupby('id')
chances = user_group.Team.value_counts()/ user_group.count().Team
crew_chances = pd.DataFrame([[f'{i[0]}', j] for i,j in list(zip(chances.index.values, chances)) if i[1] == 'Crewmate'], columns = ['id', 'Crewmate'])
imposter_chances = pd.DataFrame([[f'{i[0]}', j] for i,j in list(zip(chances.index.values, chances)) if i[1] == 'Imposter'], columns = ['id', 'Imposter'])

base = user.sort_values('Game Completed Date')['Game Completed Date']
base = base[base.index[-1]]
min_play_per_day = 5
print(base, min_play_per_day)
user['Days'] = user.sort_values('Game Completed Date')['Game Completed Date'].apply(lambda x: base.date() - x.date()).dt.days
user_date = user.set_index(['id', 'Days'])

user_date = user_date.sort_index()
#print(user_date.index.is_lexsorted())
time_based_chances = [[id_, i, user_date.loc[(id_,i)].shape[0]] + list(user_date.loc[(id_,i)].Team.value_counts().values) for id_ in user.id.unique() for i in user_date.loc[(id_)].index.value_counts()[user_date.loc[(id_)].index.value_counts() > min_play_per_day].index.to_list()]
time_based_chances = pd.DataFrame(time_based_chances, columns = ['id', 'days', 'Times', 'Crewmate', 'Imposter'])
time_based_chances.Imposter = time_based_chances.Imposter.fillna(0)
time_based_chances['Crewmate_Chance'] = time_based_chances.apply(lambda x: x['Crewmate']/( x['Crewmate'] + x['Imposter']), axis = 1)
time_based_chances['Imposter_Chance'] = time_based_chances.apply(lambda x: x['Imposter']/( x['Crewmate'] + x['Imposter']), axis = 1)

def Chance_function(data, feature, title):
    fig = make_subplots(rows = 2, cols = 1)

    bins = 10
    #feature = 'Imposter Kills'
    win = {}
    temp = data.query(f''' Outcome == "Win" ''')
    for i in temp[feature].unique():
        win[f'Win: {i}'] = plt.hist(temp[temp[feature] == i]['Game Length'], bins = bins)[0]

    loss = {}
    temp = data.query(f''' Outcome == "Loss" ''')
    for i in temp[feature].unique():
        loss[f'Loss: {i}'] = plt.hist(temp[temp[feature] == i]['Game Length'], bins = bins)[0]
    plt.close()

    for i in win.keys():
        fig.add_trace(go.Scatter(y=np.array(win[i]*100/sum(win.values())), fill='tonexty',name = f'{i}', mode= 'none'), row = 1, col = 1)

    for i in loss.keys():
        fig.add_trace(go.Scatter(y=np.array(loss[i]*100/sum(loss.values())), fill='tonexty',name = f'{i}', mode= 'none'), row = 2, col = 1)

    fig.update_xaxes(title_text='Time Bins')
    fig.update_yaxes(title_text='Percentage')
    fig.update_layout(title = title)
    fig.show()
    
def Box_function(data, feature, title):
    fig = make_subplots(rows = 1, cols = 2)
    
    temp = data.query(f''' Outcome == "Win" ''')
    for i in temp[feature].unique():
        fig.add_trace(go.Box(y=temp[temp[feature] == i]['Game Length'],name = f'{i}'), row = 1, col = 1)
    temp = data.query(f''' Outcome == "Loss" ''')
    for i in temp[feature].unique():
        fig.add_trace(go.Box(y=temp[temp[feature] == i]['Game Length'],name = f'{i}'), row = 1, col = 2)

    fig.update_xaxes(title_text='Time Bins')
    fig.update_yaxes(title_text='Percentage')
    fig.update_layout(title = title)
    fig.show()

<img src="https://www.citrusbits.com/wp-content/uploads/2020/11/123777854_2773327262914469_3127875734137888976_o-1024x473.jpg"> 
# <p style = 'text-align: center; background-color:#c51111;'> Chances of being an Imposter </p>

### <p style = 'text-align: center; background-color:#38fedc;'> Based on Users Records </p>

In [ ]:
px.bar(crew_chances.merge(imposter_chances, how = 'inner', left_on = 'id', right_on = 'id'), x = 'id', y = ['Crewmate', 'Imposter'])

In [ ]:
fig = ff.create_distplot([crew_chances['Crewmate'], imposter_chances['Imposter']], ['Crewmate', 'Imposter'], bin_size=.03)
fig.update_layout(title_text = 'Distribution of Chances')

In [ ]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Box(y = crew_chances.Crewmate, name =  'Crewmate'), row=1, col=1)
fig.add_trace(go.Box(y = imposter_chances.Imposter, name = 'Imposter'),row=1, col=2)

fig.update_layout(height=600, width=800, title_text="Chances of a Player getting either Crewmate or Imposter")
fig.show()

### <p style = 'text-align: center; background-color:#38fedc;'> Based on number of times a user played the game in a day </p>

In [ ]:
fig = px.bar(time_based_chances, y = ['Crewmate_Chance', 'Imposter_Chance'], hover_data = ['id','days', 'Times','Crewmate', 'Imposter'])
fig.update_layout(title_text = 'Chances each time a Player got either of them.')

In [ ]:
fig = ff.create_distplot([time_based_chances['Crewmate_Chance'], time_based_chances['Imposter_Chance']], group_labels = ['Crewmate', 'Imposter'], bin_size=.03)
fig.update_layout(title_text = 'Distribution of Chances')

In [ ]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Box(y = time_based_chances.Crewmate_Chance, name =  'Crewmate'), row=1, col=1)
fig.add_trace(go.Box(y = time_based_chances.Imposter_Chance, name =  'Imposter'), row=1, col=2)

fig.update_layout(height=600, width=800, title_text="Chances of a Player getting either Crewmate or Imposter")
fig.show()

# <p style = 'text-align: center; background-color:#c51111;'> Being the SUS </p>

In [ ]:
bins = 10
imposter_sus = plt.hist(imposter.query('Ejected == "Yes"')['Game Length'], bins = bins)
imposter_all = plt.hist(imposter['Game Length'], bins= bins)
crewmate_sus = plt.hist(crewmate.query('Ejected == "Yes"')['Game Length'], bins = bins)
crewmate_all = plt.hist(crewmate['Game Length'], bins = bins)
plt.close()

fig = go.Figure()
fig.add_trace(go.Scatter(y=np.array((imposter_sus[0])/(imposter_all[0])* 100), fill='tozeroy', name = 'Chance of SUS - Imposter', mode = 'none'))
fig.add_trace(go.Scatter(y=np.array((crewmate_sus[0])/(crewmate_all[0])*100), fill='tonexty',name = 'Chance of SUS - Crewmate', mode= 'none'))

fig.update_xaxes(title_text='Time Bins')
fig.update_yaxes(title_text='Percentage')
fig.update_layout(title = 'Chance of a Player getting suspected through different game length')
fig.show()

# <p style = 'text-align: center; background-color:#50ef39;'> Help for the Greater Good </p>
### <p style = 'text-align: center; background-color:#38fedc;'>Average time to Complete Tasks</p>

In [ ]:
temp = crewmate[crewmate.apply(lambda x: True if x['All Tasks Completed'] == 'Yes' else False, axis = 1)]
temp = temp.loc[:,['Task Completed', 'Time to complete all tasks']].sort_values('Task Completed', axis = 0)
fig = px.box(temp, y = 'Time to complete all tasks', color = 'Task Completed')
fig.update_xaxes(title_text = 'No of Tasks')
fig.show()

### <p style = 'text-align: center; background-color:#38fedc;'>Getting Status Vet or Rookie</p>

In [ ]:
Chance_function(crewmate, 'Rank Change', 'Crewmate Rank change with game length')

### <p style = 'text-align: center; background-color:#38fedc;'>Check your time to shine</p>

In [ ]:
Box_function(crewmate, 'Rank Change', 'Best Time for a Crewmate to get a rank change with game length')

![Getting Murdered](https://www.icegif.com/wp-content/uploads/among-us-icegif-1.gif)
### <p style = 'text-align: center; background-color:#38fedc;'>Getting Murdered</p> 

In [ ]:
fig = px.sunburst(crewmate, path=['Outcome','Murdered'])
fig.show()

# <p style = 'text-align: center; background-color:#c51111;'> Be less of a Sus </p>
### <p style = 'text-align: center; background-color:#ef7d0d;'> Being a serial killer </p>

In [ ]:
Chance_function(imposter, 'Rank Change', 'Chance of an Imposter Status Upgrade or Degrade Chances with different game length')

In [ ]:
Box_function(imposter, 'Rank Change', 'Best Time for an Imposter to get a rank change with game length')

### <p style = 'text-align: center; background-color:#ef7d0d;'> Eliminated vs Kills </p>

In [ ]:
fig = px.sunburst(imposter, path=['Outcome','Ejected', 'Imposter Kills'])
fig.show()

### <p style = 'text-align: center; background-color:#ef7d0d;'> Best Time to Strike </p>

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(x = imposter.query("""Outcome == 'Win'""")['Imposter Kills'], name = 'Win'))
fig.add_trace(go.Histogram(x = imposter.query("""Outcome == 'Loss'""")['Imposter Kills'], name = 'Loss'))

fig.update_layout(height=600, width=800, title_text="Imposter Kills")
fig.show()

In [ ]:
Chance_function(imposter, 'Imposter Kills', 'Chance of an Imposter getting a rank change with different game length on winning or losing the game')

In [ ]:
Box_function(imposter, 'Imposter Kills', 'Best Time for an Imposter to kill')

![](https://img.wattpad.com/bce35ddb88ed74be6b3c9229d858c921087560de/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f776174747061642d6d656469612d736572766963652f53746f7279496d6167652f48786a334d7446493663577754513d3d2d3134372e313634326361323338323832626633313338323932333935393938352e676966)
### <p style = 'text-align: center; background-color:#6b2fbb; color:#f5f557;'> Stay Tuned for Suggestions </p>